# Tests

## Create a stream in redis using Aia Utilities

In [1]:
from zoneinfo import ZoneInfo
import aia_utilities as au
from datetime import datetime
import pytz

ru = au.RedisUtilities()
tm = au.TimeManagement()

ru.start()

# stream = 'test_stream'
stream = 'prices'



✅ Redis started


In [2]:
ru.delete('prices')
ru.delete('algos')
ru.delete('state')
ru.delete('decisions')
# ru.delete('transactions')

False

## Populate it with n entries ordered or random timestamps

In [ ]:
from time import sleep
from datetime import datetime, timedelta
import pytz  
import random

ordered = False
iterations = 100

for i in range(iterations):

    if ordered:
        timestamp = tm.datetime_to_string(datetime.now())
    else:
        timestamp = tm.datetime_to_string(random.uniform(datetime.now() + timedelta(minutes=5), datetime.now() - timedelta(minutes=5)))
    item1 = {"timestamp": timestamp, "price": 1.0 + i, "currency": "USD"}
    ru.write(stream, item1)

## Show the stream

In [13]:
sample = 3
ru.show('trx', sample)

The stream is empty.
0 items, type <class 'list'>
Adjusted sample from 3 to 1 because stream is 0 items.
First 1:
Last 1:


In [28]:
a = ru.get_latest(stream, 'instrument', 'WTICO_USD')
a

{'timestamp': '2025-10-17 11:10:18.773993',
 'instrument': 'WTICO_USD',
 'price': 57.611,
 'bid': 57.596,
 'ask': 57.626,
 'spread': 0.03,
 'l': True}

## Read all entries and order them or not

In [44]:
stream = 'algos'

ordered = True  # False

return_dict = ru.read_all(stream, order=ordered)

is_ordered = all(return_dict[i]["timestamp"] <= return_dict[i + 1]["timestamp"] for i in range(len(return_dict) - 1))
print(f"{len(return_dict)} items, ordered: {is_ordered}")

# count items in return_dict per instrument
from collections import Counter
instruments = [entry['instrument'] for entry in return_dict]
instrument_counts = Counter(instruments)
print(instrument_counts)


# # Keep only timestamp and price and base_ema and base_tema
# for entry in return_dict:
#     keys_to_keep = ['timestamp', 'price', 'base_ema', 'base_tema']
#     for key in list(entry.keys()):
#         if key not in keys_to_keep:
#             del entry[key]

# # save entries to csv for debugging without using aia_utilities
# import pandas as pd
# pd.DataFrame(return_dict).to_csv(f'debug.csv', index=False)

23545 items, ordered: True
Counter({'XAG_USD': 16234, 'USD_CAD': 3695, 'WTICO_USD': 3616})


# Delete the stream

In [1]:
ru.delete('algos')

NameError: name 'ru' is not defined

## Test read_each by inserting items randomly and reading them

In [ ]:
# While you read each, also insert some entries once every 10 seconds, in a separate thread
import threading
from time import sleep    
import random

ru.delete('test_stream')
def insert_entries():
    # make the range 0-5 random
    while True:
        sleep(random.randint(0, 5))
        # create a now timestamp with 6 decimals for new york time
        now = datetime.now(pytz.timezone("America/New_York"))
        timestamp = now.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        item1 = {"timestamp": timestamp, "price": random.randint(0, 100), "currency": "USD"}
        ru.write(stream, item1)

# While you read each, also insert some entries once every 10 seconds, in a separate thread
thread = threading.Thread(target=insert_entries)
thread.start()

for entry in ru.read_each(stream):
    print(entry)

In [ ]:
entries = ru.read_all('prices')

# filter entries for the given instrument
entries = [e for e in entries if e.get('instrument') == 'WTICO_USD']


In [ ]:
print(len(entries))
for entry in entries:
    print(entry)


In [ ]:
# example only
import json

resp = ru.redis_db.xrange('prices')  # or xread result
for entry_id, fields in resp:
    # decode id bytes -> str
    if isinstance(entry_id, bytes):
        entry_id = entry_id.decode()

    # get the data field (may be bytes)
    raw = fields.get(b'data') or fields.get('data')
    if isinstance(raw, bytes):
        raw = raw.decode()

    # parse json and read timestamp
    data = json.loads(raw)
    ts = data.get('timestamp')

    print("stream id:", entry_id, "timestamp:", ts)